In [13]:
import numpy as np
import pandas as pd

In [14]:
worms_kmri_sq_link06_all_subjects = pd.read_csv('/home/anastasis/EMC_Thesis/OAI Data/OAICompleteData_ASCII/kmri_sq_worms_link06.txt', sep="|")
worms_kmri_sq_link06_all_subjects.rename(columns={'side':'SIDE'},inplace=True)

In [15]:
moaks_sq06_all_subjects = pd.read_csv('/home/anastasis/EMC_Thesis/My_codes/MOAKS/MOAKS_SQ06/Whole_Knee_Approach/moaks_sq06_all_info.csv',index_col=0)
moaks_sq06_all_subjects_ID_SIDE = moaks_sq06_all_subjects.loc[:,['ID','SIDE']]


In [16]:
# HERE WE KEEP ONLY THE IDs THAT ARE PRESENT ONLY IN THE WORMS STUDY AND NOT IN THE FNIH_SQ AND SQ

worms_kmri_sq_link06_with_sq = pd.merge(worms_kmri_sq_link06_all_subjects,moaks_sq06_all_subjects_ID_SIDE,on=['ID','SIDE'],how='left',indicator=True)
print(worms_kmri_sq_link06_with_sq._merge.value_counts())
worms_kmri_sq_link06_without_sq = worms_kmri_sq_link06_with_sq.loc[worms_kmri_sq_link06_with_sq._merge == 'left_only']
print(worms_kmri_sq_link06_without_sq._merge.value_counts())
worms_kmri_sq_link06_without_sq =  worms_kmri_sq_link06_without_sq.rename(columns={'_merge':'_merge_without_fnih'})


left_only     287
both           13
right_only      0
Name: _merge, dtype: int64
left_only     287
right_only      0
both            0
Name: _merge, dtype: int64


In [17]:
worms_kmri_sq_link06 = worms_kmri_sq_link06_without_sq

In [18]:
print(worms_kmri_sq_link06.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 287 entries, 0 to 299
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   ID                   287 non-null    int64   
 1   SIDE                 287 non-null    int64   
 2   VERSION              287 non-null    float64 
 3   READPRJ              287 non-null    int64   
 4   V06MWMTMA            287 non-null    int64   
 5   V06MWMTMB            287 non-null    int64   
 6   V06MWMTMP            287 non-null    int64   
 7   V06MWMTLA            287 non-null    int64   
 8   V06MWMTLB            287 non-null    int64   
 9   V06MWMTLP            287 non-null    int64   
 10  V06MWMTMS            287 non-null    int64   
 11  V06MWMTLS            287 non-null    int64   
 12  V06MWACL             287 non-null    int64   
 13  V06MWPCL             287 non-null    int64   
 14  V06MWMCL             287 non-null    int64   
 15  V06MWLCL             28

# WORMS_OA_FEATURES_THRES1 
# Cartilage grade >1 AND BML>1 AND Meniscus Degradation > 1

In [19]:
# FIRST WE CONSTRUCT A NEW DATAFRAME WITH ALL THE OA FEATURES AS COLUMNS

# THE FIRST FEATURE DATAFRAME WE CONSTRUCT IS WITH THRESHOLD > 1 
# FOR BOTH BML AND CARTILAGE THICKNESS LOSS 
# Because it is not clear yet which threshold to use (1 or 0) 

worms_OA_features_thres1 = pd.DataFrame(
    columns=['ID','SIDE','Cartilage_Morphology','BML', 'Meniscus_Degradation'])
# the ID and SIDE columns are integers
worms_OA_features_thres1[['ID','SIDE']] = worms_kmri_sq_link06[['ID','SIDE']]
# and we fill the rest of the df with np.nan values
worms_OA_features_thres1.iloc[:,2:] = np.nan

print('Type of ID and SIDE columns {} and type of the rest of the columns {}'
    .format(type(worms_OA_features_thres1.iloc[0,0]),type(worms_OA_features_thres1.iloc[0,2])))


Type of ID and SIDE columns <class 'numpy.int64'> and type of the rest of the columns <class 'numpy.float64'>


In [20]:
worms_OA_features_thres1.loc[((worms_kmri_sq_link06.V06MWCMFM > 1) | (worms_kmri_sq_link06.V06MWCMFL > 1) |
                (worms_kmri_sq_link06.V06MWCMTM > 1) | (worms_kmri_sq_link06.V06MWCMTL > 0)),'Cartilage_Morphology'] = 1

worms_OA_features_thres1.loc[((worms_kmri_sq_link06.V06MWBMFM > 1) | (worms_kmri_sq_link06.V06MWBMFL > 1) |
                (worms_kmri_sq_link06.V06MWBMTM > 1) | (worms_kmri_sq_link06.V06MWBMTL > 1)),'BML'] = 1

worms_OA_features_thres1.loc[((worms_kmri_sq_link06.V06MWMTMS > 1) | (worms_kmri_sq_link06.V06MWMTLS > 1)),'Meniscus_Degradation'] = 1                

worms_OA_features_thres1.fillna(0, inplace=True)


In [21]:
worms_cart_with_OA_thres1 = worms_OA_features_thres1.loc[((worms_OA_features_thres1.Cartilage_Morphology == 1))]
print('Threshold 1 - Number of subjects with worms grade for cartilage morphology larger than ONE : ',
        format(worms_OA_features_thres1.Cartilage_Morphology.eq(1).sum()))
worms_cart_with_OA_thres1.to_csv('worms_sq06_cartilage_with_OA_thres1.csv')        

worms_bml_with_OA_thres1 = worms_OA_features_thres1.loc[((worms_OA_features_thres1.BML == 1))]        
print('Threshold 1 - Number of subjects with worms grade for BML larger than ONE : ',
        format(worms_OA_features_thres1.BML.eq(1).sum()))
worms_bml_with_OA_thres1.to_csv('worms_sq06_bml_with_OA_thres1.csv')

worms_men_with_OA_thres1 = worms_OA_features_thres1.loc[((worms_OA_features_thres1.Meniscus_Degradation == 1))]
print('Threshold 1 - Number of subjects with worms grade for Meniscus Degradation larger than ONE : ',
        format(worms_OA_features_thres1.Meniscus_Degradation.eq(1).sum()))
worms_men_with_OA_thres1.to_csv('worms_sq06_men_with_OA_thres1.csv')

Threshold 1 - Number of subjects with worms grade for cartilage morphology larger than ONE :  189
Threshold 1 - Number of subjects with worms grade for BML larger than ONE :  49
Threshold 1 - Number of subjects with worms grade for Meniscus Degradation larger than ONE :  76


In [22]:
worms_wout_cart_thres1 = worms_OA_features_thres1.loc[(worms_OA_features_thres1.Cartilage_Morphology == 0)]                    
print('Number of subjects WITHOUT Cartilage is : ',format(worms_wout_cart_thres1.shape[0]))        
worms_wout_cart_thres1.to_csv('worms_sq06_without_Cart_thres1.csv')

worms_wout_bml_thres1 = worms_OA_features_thres1.loc[(worms_OA_features_thres1.BML == 0)]                    
print('Number of subjects WITHOUT BML is : ',format(worms_wout_bml_thres1.shape[0]))        
worms_wout_bml_thres1.to_csv('worms_sq06_without_BML_thres1.csv')

worms_wout_men_thres1 = worms_OA_features_thres1.loc[(worms_OA_features_thres1.Meniscus_Degradation == 0)]                    
print('Number of subjects WITHOUT OA is : ',format(worms_wout_men_thres1.shape[0]))        
worms_wout_men_thres1.to_csv('worms_sq06_without_Meniscus_thres1.csv')

Number of subjects WITHOUT Cartilage is :  98
Number of subjects WITHOUT BML is :  238
Number of subjects WITHOUT OA is :  211


In [23]:
worms_sq06_all_info_thres1 = worms_OA_features_thres1.merge(worms_kmri_sq_link06,on=['ID','SIDE'], how='inner')
worms_sq06_all_info_thres1.to_csv('worms_sq06_all_info_thres1.csv')


# WORMS_OA_FEATURES_THRES0
# Cartilage grade >0 AND BML>0 AND Meniscus Degradation > 0

In [24]:
worms_OA_features_thres0 = pd.DataFrame(
    columns=['ID','SIDE','Cartilage_Morphology','BML', 'Meniscus_Degradation'])
# the ID and SIDE columns are integers
worms_OA_features_thres0[['ID','SIDE']] = worms_kmri_sq_link06[['ID','SIDE']]
# and we fill the rest of the df with np.nan values
worms_OA_features_thres0.iloc[:,2:] = np.nan

print('Type of ID and SIDE columns {} and type of the rest of the columns {}'
    .format(type(worms_OA_features_thres0.iloc[0,0]),type(worms_OA_features_thres0.iloc[0,2])))


Type of ID and SIDE columns <class 'numpy.int64'> and type of the rest of the columns <class 'numpy.float64'>


In [25]:
worms_OA_features_thres0.loc[((worms_kmri_sq_link06.V06MWCMFM > 0) | (worms_kmri_sq_link06.V06MWCMFL > 0) |
                (worms_kmri_sq_link06.V06MWCMTM > 0) | (worms_kmri_sq_link06.V06MWCMTL > 0)),'Cartilage_Morphology'] = 1

worms_OA_features_thres0.loc[((worms_kmri_sq_link06.V06MWBMFM > 0) | (worms_kmri_sq_link06.V06MWBMFL > 0) |
                (worms_kmri_sq_link06.V06MWBMTM > 0) | (worms_kmri_sq_link06.V06MWBMTL > 1)),'BML'] = 1

worms_OA_features_thres0.loc[((worms_kmri_sq_link06.V06MWMTMS > 0) | (worms_kmri_sq_link06.V06MWMTLS > 0)),'Meniscus_Degradation'] = 1                

worms_OA_features_thres0.fillna(0, inplace=True)


In [26]:
worms_cart_with_OA_thres0 = worms_OA_features_thres0.loc[((worms_OA_features_thres0.Cartilage_Morphology == 1))]
print('Threshold 0 - Number of subjects with worms grade for cartilage morphology larger than ZERO : ',
        format(worms_OA_features_thres0.Cartilage_Morphology.eq(1).sum()))
worms_cart_with_OA_thres0.to_csv('worms_sq06_cartilage_with_OA_thres0.csv')        

worms_bml_with_OA_thres0 = worms_OA_features_thres0.loc[((worms_OA_features_thres0.BML == 1))]        
print('Threshold 0 - Number of subjects with worms grade for BML larger than ZERO : ',
        format(worms_OA_features_thres0.BML.eq(1).sum()))
worms_bml_with_OA_thres0.to_csv('worms_sq06_bml_with_OA_thres0.csv')

worms_men_with_OA_thres0 = worms_OA_features_thres0.loc[((worms_OA_features_thres0.Meniscus_Degradation == 1))]
print('Threshold 0 - Number of subjects with worms grade for Meniscus Degradation larger than ZERO : ',
        format(worms_OA_features_thres0.Meniscus_Degradation.eq(1).sum()))
worms_men_with_OA_thres0.to_csv('worms_sq06_men_with_OA_thres0.csv')

Threshold 0 - Number of subjects with worms grade for cartilage morphology larger than ZERO :  197
Threshold 0 - Number of subjects with worms grade for BML larger than ZERO :  65
Threshold 0 - Number of subjects with worms grade for Meniscus Degradation larger than ZERO :  128


In [27]:
worms_wout_cart_thres0 = worms_OA_features_thres0.loc[(worms_OA_features_thres0.Cartilage_Morphology == 0)]                    
print('Number of subjects WITHOUT Cartilage is : ',format(worms_wout_cart_thres0.shape[0]))        
worms_wout_cart_thres0.to_csv('worms_sq06_without_Cart_thres0.csv')

worms_wout_bml_thres0 = worms_OA_features_thres0.loc[(worms_OA_features_thres0.BML == 0)]                    
print('Number of subjects WITHOUT BML is : ',format(worms_wout_bml_thres0.shape[0]))        
worms_wout_bml_thres0.to_csv('worms_sq06_without_BML_thres0.csv')

worms_wout_men_thres0 = worms_OA_features_thres0.loc[(worms_OA_features_thres0.Meniscus_Degradation == 0)]                    
print('Number of subjects WITHOUT OA is : ',format(worms_wout_men_thres0.shape[0]))        
worms_wout_men_thres0.to_csv('worms_sq06_without_Meniscus_thres0.csv')

Number of subjects WITHOUT Cartilage is :  90
Number of subjects WITHOUT BML is :  222
Number of subjects WITHOUT OA is :  159


In [28]:
worms_sq06_all_info_thres0 = worms_OA_features_thres0.merge(worms_kmri_sq_link06,on=['ID','SIDE'], how='inner')
worms_sq06_all_info_thres0.to_csv('worms_sq06_all_info_thres0.csv')
